Edwin Sanchez

# Anomaly Detection

This notebook contains code exemplifying how to do anomaly detection using an example dataset.

> I "borrowed" a lot of code from the PyTorch Metric Learning's [Examples on Google Colab](https://github.com/KevinMusgrave/pytorch-metric-learning/blob/master/examples/README.md), specifically the [MNIST using SubCenterArcFaceLoss](https://github.com/KevinMusgrave/pytorch-metric-learning/blob/master/examples/notebooks/SubCenterArcFaceMNIST.ipynb) Notebook.

In [ ]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

## Additive Angular Margin Loss

The next section contains an implementation of the loss function called "Additive Angular Margin Loss", which is a loss function designed to create feature embeddings in a space of a target size. It does this by forcing output embeddings of a certain class to be *closer* in euclidean space, all while forcing embeddings of different classes to be farther apart. It does this along the surface of a hypersphere, with dimensions equal to the number of embedding dimensions (output dimensionality). For more details, please see the Acknowledgements section of the README, which contains a reference to ArcFace, the paper that introduces this loss function.

## Read in Dataset

## Define Useful Functions

## Implement the Model

## Train Model